## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [2]:
import os
from tensorflow import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
from tensorflow.keras.layers import BatchNormalization

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [8]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from tensorflow.keras.callbacks import ModelCheckpoint

model_ckpt_true = ModelCheckpoint(filepath="./tmpt.h5", 
                             monitor="val_loss", 
                             save_best_only=True)
model_ckpt_false = ModelCheckpoint(filepath="./tmpf.h5", 
                             monitor="val_loss", 
                             save_best_only=False)
MODEL_CKPT=[model_ckpt_true,model_ckpt_false]
FILEPATH=["./tmpt.h5","./tmpf.h5"]
results = {}
loss_loadback = {}
acc_loadback = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for model_ckpt,filepath in zip(MODEL_CKPT,FILEPATH):
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[model_ckpt])

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    exp_name_tag = "SaveBest-%s" % str(save)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}
    # Load back
    model = keras.models.load_model(filepath)
    loss_loadback[exp_name_tag], acc_loadback[exp_name_tag] = model.evaluate(x_test, y_test)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_v2 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 128)               512   

50000/50000 [==============================] - 7s 139us/sample - loss: 0.5876 - accuracy: 0.8223 - val_loss: 1.6021 - val_accuracy: 0.4967
Epoch 46/50
50000/50000 [==============================] - 7s 137us/sample - loss: 0.5717 - accuracy: 0.8308 - val_loss: 1.5947 - val_accuracy: 0.4914
Epoch 47/50
50000/50000 [==============================] - 7s 136us/sample - loss: 0.5527 - accuracy: 0.8382 - val_loss: 1.6563 - val_accuracy: 0.4864
Epoch 48/50
50000/50000 [==============================] - 7s 148us/sample - loss: 0.5376 - accuracy: 0.8448 - val_loss: 1.6317 - val_accuracy: 0.4872
Epoch 49/50
50000/50000 [==============================] - 7s 142us/sample - loss: 0.5216 - accuracy: 0.8483 - val_loss: 1.6268 - val_accuracy: 0.4901
Epoch 50/50
50000/50000 [==============================] - 7s 138us/sample - loss: 0.5029 - accuracy: 0.8575 - val_loss: 1.6674 - val_accuracy: 0.4867


NameError: name 'save' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["y", "b"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
    plt.hlines(y=loss_loadback[cond], xmin=0, xmax=len(train_loss), colors=color_bar[i], linestyles='--')
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
    plt.hlines(y=acc_loadback[cond], xmin=0, xmax=len(train_loss), colors='r', linestyles='--')
plt.title("Accuracy")
plt.legend()
plt.show()